# MNIST Tutorial

In this tutorial we will be learning how to use NeuroTorch to train a neural network to recognize handwritten digits.

## Setup

You can now install the dependencies by running the following commands:

In [ ]:
!pip install -r mnist_requirements.txt
!pip install norse

If you have a cuda device and want to use it for this tutorial (it is recommended to do so), you can uninstall pytorch with `pip uninstall torch` and re-install it with the right cuda version by generating a command with [PyTorch GetStarted](https://pytorch.org/get-started/locally/) web page.

After setting up the virtual environment, we will need to import the necessary packages.

In [ ]:
import os
import pprint
from collections import OrderedDict

import psutil
import torch
from torchvision.transforms import Compose
import norse
from pythonbasictools.device import log_device_setup, DeepLib
from pythonbasictools.logging import logs_file_setup

from tutorials.mnist.dataset import get_dataloaders, DatasetId
import neurotorch as nt
from neurotorch import Dimension, DimensionProperty
from neurotorch.callbacks import CheckpointManager, LoadCheckpointMode
from neurotorch.metrics import ClassificationMetrics
from neurotorch.trainers import ClassificationTrainer
from neurotorch.transforms.spikes_encoders import SpyLIFEncoder

In [ ]:
logs_file_setup("mnist_tutorial", add_stdout=False)
log_device_setup(deepLib=DeepLib.Pytorch)
if torch.cuda.is_available():
	torch.cuda.set_per_process_memory_fraction(0.8)

## Initialization

It's the time to defined our training and networks parameters. In the following,

- the number of iteration is the number of time the trainer will pass through the entire training dataset;
- the batch size is the number of samples that will be loaded at the same time for each forward and backward pass;
- the learning rate is the learning rate used by the optimizer;
- the number of steps is the number of euler integration steps performed by the model during each forward and backward pass;
- the number of hidden neurons is the number of neurons that will be used in the hidden layer of the model;
- dt is the time step of the euler integration;

In [ ]:
dataset_id = DatasetId.MNIST

# Training parameters
n_iterations = 30
batch_size = 1024
learning_rate = 1e-3

# Network parameters
n_steps = 8
n_hidden_neurons = 128
dt = 1e-3

Here we're initializing a callback of the trainer used to save the network during the training.

In [ ]:
checkpoint_folder = f"./checkpoints/{dataset_id.name}"
checkpoint_manager = CheckpointManager(checkpoint_folder, save_best_only=True)

NeuroTorch is a library that allows to build bio-like networks, that is, they are in general recurrent neural networks. In this task we are classifying images, so we must transform the input images into times series of values. In our case, we are using spiking neural network to perform the classification, so we will transform the input images into spikes trains. The used transform in this tutorial is the ConstantCurrentLIFEncoder from the [norse](https://github.com/norse/norse) library.

In [ ]:
# input_transform = Compose([torch.nn.Flatten(start_dim=1), norse.torch.ConstantCurrentLIFEncoder(seq_length=n_steps, dt=dt), Lambda(lambda x: x.permute(1, 0, 2))])
input_transform = Compose([torch.nn.Flatten(start_dim=2), SpyLIFEncoder(n_steps=n_steps, n_units=28*28)])

It is time to load the dataloaders. The dataloader is an object used to the data from a given dataset. In our case we will load the MNIST or the FASHION-MNIST dataset. To change the dataset, just change the value of the variable `dataset_id` for `DatasetId.MNIST` or `DatasetId.FASHION_MNIST` in the cell below.


Here is an exemple of the MNIST dataset:
<p align="center"> <img width="1200" height="500" src="../../images/mnist/MnistExamples.png"> </p>

And an exemple of the Fashion-MNIST dataset:
<p align="center"> <img width="1200" height="500" src="../../images/mnist/fashion-mnist-sprite.png"> </p>

In [ ]:
dataloaders = get_dataloaders(
	dataset_id=dataset_id,
	batch_size=batch_size,
	train_val_split_ratio=0.95,
	nb_workers=max(0, min(2, psutil.cpu_count(logical=False))),
	pin_memory=True,
)

We can finally define our model. The sequential model is a neural network that is composed of a list of layers. The list of layers is ordered and the first layer is the input layer. The last layer is the output layer. The sequential model can be found in the subpackage `neurotorch.modules`. Note that we can specify the input transform that will be applied to the input data before the forward pass through the list of layers. If the transformation is computationally expensive, it is recommended to put it in the dataloaders instead of the model for the training. Usually the input transform in the model is used when the model is used for inference.

In [ ]:
network = nt.SequentialRNN(
	input_transform=input_transform,
	layers=[
		nt.LIFLayer(
			input_size=[Dimension(None, DimensionProperty.TIME), Dimension(28*28, DimensionProperty.NONE)],
			use_recurrent_connection=False,
			output_size=n_hidden_neurons,
			dt=dt,
		),
		nt.SpyLILayer(dt=dt, output_size=10),
	],
	name=f"{dataset_id.name}_network",
	checkpoint_folder=checkpoint_folder,
	hh_memory_size=1,
)

After the instantiation of the model, we have to build it, so it can infer the missing sizes of the layers in the model and create the tensors that will be used during the training.

In [ ]:
network.build()
print(network)

## Training

In the next cell, we create the trainer that will be used to trained our network. They are several types of trainers that can be found in the subpackage `neurotorch.trainers` like the `ClassificationTrainer`, the `RegressionTrainer` or the `PPOTrainer`. In this tutorial, we will use the `ClassificationTrainer` because we are trying to classify some images. Note that the callbacks are some object that will be called during the training. They are found in the subpackage `neurotorch.callbacks` like the `CheckpointManager`.

In [ ]:
trainer = ClassificationTrainer(
	model=network,
	optimizer=torch.optim.Adam(network.parameters(), lr=learning_rate),
	callbacks=checkpoint_manager,
	verbose=True,
)
print(trainer)

The training will start! Let's see how our simple network will perform on this task.

Note: the argument `force_overwrite` is used to overwrite the existing checkpoint if it exists and the argument `load_checkpoint_mode` if the model will be loaded from the last checkpoint (`LoadCheckpointMode.LAST_ITR`) or from the best checkpoint (`LoadCheckpointMode.BEST_ITR`). The loading mode is really useful when you want to stop the training and restart it later.

In [ ]:
training_history = trainer.train(
	dataloaders["train"],
	dataloaders["val"],
	n_iterations=n_iterations,
	load_checkpoint_mode=LoadCheckpointMode.LAST_ITR,
	# force_overwrite=True,
)

The training is finished! Let's see how our network learned over the iterations.

Note: the next graphs are really useful to see if the model overfit or underfit the data over the iterations. It's why you can use the callback `TrainingHistoryVisualizationCallback` to see those graphs in real time.

In [ ]:
training_history.plot(show=True)

We can load the model at the best or the last iteration for the testing phase.

In [ ]:
network.load_checkpoint(checkpoint_manager.checkpoints_meta_path, LoadCheckpointMode.BEST_ITR, verbose=True)

We can now see how our network perform on the test dataset.

In [ ]:
accuracies={
	k: ClassificationMetrics.accuracy(network, dataloaders[k], verbose=True, desc=f"{k}_accuracy")
	for k in dataloaders
}
pprint.pprint(accuracies)

Let's see the results on other popular classification metrics.

In [ ]:
precisions={
	k: ClassificationMetrics.precision(network, dataloaders[k], verbose=True, desc=f"{k}_precision")
	for k in dataloaders
},
recalls={
	k: ClassificationMetrics.recall(network, dataloaders[k], verbose=True, desc=f"{k}_recall")
	for k in dataloaders
},
f1s={
	k: ClassificationMetrics.f1(network, dataloaders[k], verbose=True, desc=f"{k}_f1")
	for k in dataloaders
}

In [ ]:
results = OrderedDict(dict(
	network=network,
	accuracies=accuracies,
	precisions=precisions,
	recalls=recalls,
	f1s=f1s,
))

In [ ]:
pprint.pprint(results, indent=4)

This is the end of the tutorial!

# Conclusion

NeuroTorch is a library that allows you to easily build neural networks and train them. Moreover, it allows you to do image classification using dynamics from neurosciences.